# RAG Pipeline with a Local Phi-3 Model and ArXiv Dataset

This notebook demonstrates how to build a complete Retrieval-Augmented Generation (RAG) system from scratch using a locally hosted `microsoft/Phi-3-mini-4k-instruct` model. The pipeline will ingest and index a subset of the ArXiv dataset, retrieve relevant academic papers based on a user's query, and generate a comprehensive answer.

**Key Steps:**
1.  **Setup**: Load the local Phi-3 model and create a text generation pipeline.
2.  **Data Loading & Preprocessing**: Download the ArXiv dataset and prepare it for indexing.
3.  **Indexing**: Split the documents, create embeddings, and store them in a FAISS vector store.
4.  **Retrieval**: Create a retriever to find relevant document chunks for a given query.
5.  **Generation**: Combine the retrieved context with the query and use the local Phi-3 model to generate an answer.
6.  **Advanced Features**: Implement an optional reranking step to improve retrieval quality.

## 1. Setup & Model Loading

In [3]:
import os
os.environ["GROQ_API_KEY"] 

'gsk_****_HIDDEN_API_KEY_****'

In [4]:
from huggingface_hub import snapshot_download

model_id = "microsoft/Phi-3-mini-4k-instruct"
model_path = snapshot_download(repo_id=model_id)


In [5]:
import os
import pandas as pd
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load the local Phi-3 model
# Make sure you have downloaded the model to this cache location first.
# Load tokenizer and model
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

# Create a text generation pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

print("Model loaded successfully!")

# Example text generation
prompt = "Once upon a time"
output = generator(prompt)
print(output[0]['generated_text'])

KeyboardInterrupt: 

## 2. Data Loading & Preprocessing

In [6]:
import kagglehub

print("Downloading dataset...")
# This requires kagglehub to be installed (pip install kagglehub) and your Kaggle API token to be set up.
path = kagglehub.dataset_download("Cornell-University/arxiv")
data_file = os.path.join(path, "arxiv-metadata-oai-snapshot.json")
print(f"Dataset file located at: {data_file}")

# Load a subset of the data to manage memory usage
def load_data_subset(file_path, num_records=50000):
    records = []
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if i >= num_records:
                break
            records.append(json.loads(line))
    return pd.DataFrame(records)

df = load_data_subset(data_file)
print(f"Successfully loaded {len(df)} records.")

# Preprocessing
df['update_date'] = pd.to_datetime(df['update_date'])
df['year'] = df['update_date'].dt.year
df = df.dropna(subset=['abstract'])
df = df[df['abstract'].str.strip() != '']

Dataset file located at: C:\Users\wasif\.cache\kagglehub\datasets\Cornell-University\arxiv\versions\250\arxiv-metadata-oai-snapshot.json
Successfully loaded 50000 records.


In [7]:
from langchain_core.documents import Document

# Create LangChain Document objects for easier processing in the RAG pipeline
documents = []
for _, row in df.iterrows():
    page_content = f"Title: {row['title']}\n\nAbstract: {row['abstract']}"
    metadata = {
        "id": row.get('id', 'N/A'),
        "authors": row.get('authors', 'N/A'),
        "year": row.get('year', 'N/A'),
        "categories": row.get('categories', 'N/A')
    }
    documents.append(Document(page_content=page_content, metadata=metadata))

print(f"Created {len(documents)} Document objects.")
print("\n--- Sample Document ---")
print(documents[0].page_content[:500])
print(f"\nMetadata: {documents[0].metadata}")

Created 50000 Document objects.

--- Sample Document ---
Title: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies

Abstract:   A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders. All
next-to-leading order perturbative contributions from quark-antiquark,
gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as
all-orders resummation of initial-state gluon radiation valid at
next-to-next-to-leading logari

Metadata: {'id': '0704.0001', 'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan", 'year': 2008, 'categories': 'hep-ph'}


## 3. The Core RAG Pipeline

### 3.1. Indexing

We will now process the loaded documents and store them in a vector database for efficient retrieval. We will use a local sentence-transformer model for creating embeddings and FAISS for the vector store.

In [ ]:
import os
import torch
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

save_path = "faiss_index"

if os.path.exists(save_path):
    print(f"Loading existing FAISS index from {save_path}...")
    embedding_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
    )
    vectorstore = FAISS.load_local(
        save_path,
        embedding_model,
        allow_dangerous_deserialization=True  # needed if pickle is used internally
    )
    print("Vector store loaded successfully.")

else:
    print("No existing FAISS index found. Creating a new one...")

    # 1. Split documents
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        add_start_index=True
    )
    all_splits = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(all_splits)} chunks.")

    # 2. Create embeddings
    embedding_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
    )

    # 3. Build vectorstore
    print("\nCreating vector store... This may take a few minutes.")
    vectorstore = FAISS.from_documents(documents=all_splits, embedding=embedding_model)
    vectorstore.save_local(save_path)
    print(f"Vector store created and saved to {save_path}.")


Loading existing FAISS index from faiss_index...


C:\Users\wasif\AppData\Local\Temp\ipykernel_12832\1314418873.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


Vector store loaded successfully.


In [18]:
# Save FAISS vectorstore to local directory
save_path = "faiss_index"
vectorstore.save_local(save_path)
print(f"Vector store saved to {save_path}")

Vector store saved to faiss_index


### 3.2. Retrieval

The retriever's job is to fetch the most relevant document chunks from the vector store based on the user's query.

In [9]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Test the retriever with a sample query
query = "What is the theory of sparsity-certifying graph decompositions?"
retrieved_docs = retriever.invoke(query)

print(f"Retrieved {len(retrieved_docs)} documents for the query: '{query}'\n")
for i, doc in enumerate(retrieved_docs):
    print(f"--- Document {i+1} ---")
    print(doc.page_content[:300] + "...")
    print(f"Metadata: {doc.metadata}\n")

Retrieved 3 documents for the query: 'What is the theory of sparsity-certifying graph decompositions?'

--- Document 1 ---
Title: Characterizing Sparse Graphs by Map Decompositions

Abstract:   A {\bf map} is a graph that admits an orientation of its edges so that each
vertex has out-degree exactly 1. We characterize graphs which admit a
decomposition into $k$ edge-disjoint maps after: (1) the addition of {\it any}
$\el...
Metadata: {'id': '0704.3843', 'authors': 'Ruth Haas, Audrey Lee, Ileana Streinu, and Louis Theran', 'year': 2011, 'categories': 'math.CO', 'start_index': 0}

--- Document 2 ---
Title: Sparsity-certifying Graph Decompositions

Abstract:   We describe a new algorithm, the $(k,\ell)$-pebble game with colors, and use
it obtain a characterization of the family of $(k,\ell)$-sparse graphs and
algorithmic solutions to a family of problems concerning tree decompositions of
graphs....
Metadata: {'id': '0704.0002', 'authors': 'Ileana Streinu and Louis Theran', 'year': 2008, 'c

### 3.3. Generation with Local Phi-3 Model

Now we'll tie everything together into a chain. The chain will:
1. Take the user's question.
2. Use the retriever to find relevant context.
3. Format the context and question into a prompt.
4. Pass the prompt to our local Phi-3 model to generate the final answer.

In [12]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_groq import ChatGroq
import os
# Initialize Groq LLM (choose model from Groq's catalog, e.g. LLaMA-3 or Mixtral)
llm = ChatGroq(
    model="meta-llama/llama-4-maverick-17b-128e-instruct",   # or "mixtral-8x7b-32768"
    temperature=0.7,
    max_tokens=512,
    groq_api_key=os.environ["GROQ_API_KEY"],
)

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

# Helper function to format docs
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# Prompt template
rag_prompt_template = """Answer the following question based on the provided context.

Context:
{context}

Question: {question}

Answer: """

prompt = PromptTemplate(
    template=rag_prompt_template,
    input_variables=["context", "question"]
)

# Build RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)


## 4. Testing the Complete RAG Pipeline

In [14]:
# Test with a list of different queries
test_queries = [
    "What is the theory of sparsity-certifying graph decompositions?",
    "Tell me about recent advances in machine learning",
    "What are the applications of quantum computing?"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    # Get the answer using the RAG chain
    answer = rag_chain.invoke(query)
    print(f"\nAnswer:\n{answer.content if hasattr(answer, 'content') else answer}")
    
    # Optionally, show the documents that were retrieved to generate the answer
    print(f"\n--- Retrieved Documents ---")
    retrieved = retriever.invoke(query)
    for i, doc in enumerate(retrieved[:2]):  # Show first 2 docs for brevity
        print(f"Doc {i+1}: {doc.page_content[:200]}...")


Query: What is the theory of sparsity-certifying graph decompositions?

Answer:
## Step 1: Understand the context of the question
The question is asking about the theory of sparsity-certifying graph decompositions, which is related to the characterization and algorithmic solutions for sparse graphs.

## Step 2: Review the provided abstracts to identify relevant information
The abstracts of the papers "Characterizing Sparse Graphs by Map Decompositions" and "Sparsity-certifying Graph Decompositions" provide insights into the characterization of sparse graphs and the algorithms used for their decomposition.

## Step 3: Identify the key elements of sparsity-certifying graph decompositions
The abstract of "Sparsity-certifying Graph Decompositions" mentions the $(k,\ell)$-pebble game with colors as a new algorithm for characterizing $(k,\ell)$-sparse graphs and solving problems related to tree decompositions.

## Step 4: Relate the $(k,\ell)$-pebble game with colors to sparsity-certifying 

## 5. Advanced RAG with Reranking (Optional)

Sometimes, the initial retrieval can be noisy. A reranking step can improve the quality of the context provided to the LLM by re-sorting the initially retrieved documents based on a more fine-grained relevance score.

In [ ]:
from typing import List

# A simple reranker class that uses the same embedding model to calculate cosine similarity
class SimpleReranker:
    def __init__(self, embedding_model):
        self.embedding_model = embedding_model
    
    def rerank(self, query: str, documents: List[Document], top_k: int = 3):
        # Get embeddings for the query
        query_embedding = self.embedding_model.embed_query(query)
        
        # Calculate similarity scores for each document
        scores = []
        for doc in documents:
            doc_embedding = self.embedding_model.embed_query(doc.page_content)
            # Simple cosine similarity calculation
            similarity = sum(a*b for a, b in zip(query_embedding, doc_embedding))
            scores.append((similarity, doc))
        
        # Sort documents by similarity score in descending order and return the top_k
        scores.sort(key=lambda x: x[0], reverse=True)
        return [doc for _, doc in scores[:top_k]]

# Create reranker instance
reranker = SimpleReranker(embedding_model)

# Enhanced retrieval function that includes a reranking step
def retrieve_and_rerank(query, k_retrieve=6, k_rerank=3):
    # Initial retrieval of a larger set of documents
    initial_docs = retriever.invoke(query)
    
    # Rerank the initially retrieved documents
    reranked_docs = reranker.rerank(query, initial_docs, top_k=k_rerank)
    
    return reranked_docs

# Test the reranking process
query = "What are neural networks used for?"
reranked_docs = retrieve_and_rerank(query)
print(f"Reranked documents for: '{query}'")
for i, doc in enumerate(reranked_docs):
    print(f"\nDoc {i+1}: {doc.page_content[:300]}...")

## 6. Save and Load the Vector Store

In [ ]:
# Save the created vector store to disk for future use, avoiding the need to re-index
vectorstore.save_local("arxiv_vectorstore")
print("Vector store saved to 'arxiv_vectorstore'")

# To load it in a future session:
# loaded_vectorstore = FAISS.load_local("arxiv_vectorstore", embedding_model, allow_dangerous_deserialization=True)
# loaded_retriever = loaded_vectorstore.as_retriever()

## 7. Batch Processing for Multiple Queries

In [ ]:
def batch_rag_query(queries: List[str]):
    """Process a list of queries in a batch and handle potential errors."""
    results = []
    for query in queries:
        try:
            answer = rag_chain.invoke(query)
            results.append({
                "query": query,
                "answer": answer,
                "status": "success"
            })
        except Exception as e:
            results.append({
                "query": query,
                "answer": None,
                "status": f"error: {str(e)}"
            })
    return results

# Test batch processing with a new set of queries
batch_queries = [
    "What is deep learning?",
    "Explain reinforcement learning",
    "What are transformers in NLP?"
]

batch_results = batch_rag_query(batch_queries)
for result in batch_results:
    print(f"\nQ: {result['query']}")
    print(f"A: {result['answer'][:200] if result['answer'] else result['status']}...")